--- INSTALASI SERPAPI & TEXTBLOB ---

In [1]:
print(" Menginstal library SerpApi...")
!pip install google-search-results -q
!pip install textblob -q
!python -m textblob.download_corpora -q

 Menginstal library SerpApi...



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already u

--- LIBRARY ---

In [6]:
import pandas as pd
from serpapi import GoogleSearch
from textblob import TextBlob

--- API KEY ---

In [7]:
API_KEY = ''

--- ID TARGET (TARGET SCARAPING) ---

In [13]:
target_data_id = "0x2e7a5835bdc19b13:0x2102f0914d826e06" # Data ID Target
nama_file_output = "ulasan_manna_kampus_simanjuntak.csv"
print(f" Target data_id: {target_data_id}")

 Target data_id: 0x2e7a59f02af0d11d:0x8b9783ed062b3931


--- MULAI SCARAPING ---

In [9]:
print("\n Melakukan permintaan awal ke SerpApi...")
params = {
    "engine": "google_maps_reviews",
    "data_id": target_data_id,
    "api_key": API_KEY,
    "lang": "id"
}

search = GoogleSearch(params)
results = search.get_dict()

# Inisialisasi list untuk menampung semua ulasan
all_reviews = []

# Cek apakah ada pesan error dari SerpApi sebelum melanjutkan
if "error" in results:
    print(f"\n TERJADI ERROR DARI SERPAPI: {results['error']}")
    print("   -> Pastikan 'data_id' sudah benar dan API Key Anda valid/memiliki kuota.")
else:
    # Jika tidak ada error, baru lanjutkan proses looping (paginasi)
    print(" Permintaan awal berhasil, memulai pengambilan ulasan...")
    page_num = 0

    # Loop ini akan otomatis mengikuti "halaman berikutnya" sampai semua ulasan habis
    while True:
        page_num += 1
        reviews_page = results.get("reviews", [])

        # Kondisi jika halaman pertama kosong meskipun tidak ada error
        if not reviews_page and page_num == 1:
            print("   -> Lokasi ditemukan, namun tidak memiliki ulasan.")
            break

        if not reviews_page:
            print(" Tidak ada ulasan lagi di halaman berikutnya.")
            break

        all_reviews.extend(reviews_page)
        print(f"  -> Halaman {page_num}: Mendapatkan {len(reviews_page)} ulasan. Total sejauh ini: {len(all_reviews)}")

        # Cek apakah ada halaman berikutnya
        if "next_page_token" not in results.get("serpapi_pagination", {}):
            print(" Semua halaman ulasan telah berhasil di-scrape.")
            break

        # Pindah ke halaman berikutnya
        params["next_page_token"] = results.get("serpapi_pagination", {}).get("next_page_token")
        search = GoogleSearch(params)
        results = search.get_dict()


 Melakukan permintaan awal ke SerpApi...
 Permintaan awal berhasil, memulai pengambilan ulasan...
  -> Halaman 1: Mendapatkan 8 ulasan. Total sejauh ini: 8
  -> Halaman 2: Mendapatkan 10 ulasan. Total sejauh ini: 18
  -> Halaman 3: Mendapatkan 10 ulasan. Total sejauh ini: 28
  -> Halaman 4: Mendapatkan 10 ulasan. Total sejauh ini: 38
  -> Halaman 5: Mendapatkan 10 ulasan. Total sejauh ini: 48
  -> Halaman 6: Mendapatkan 10 ulasan. Total sejauh ini: 58
  -> Halaman 7: Mendapatkan 10 ulasan. Total sejauh ini: 68
  -> Halaman 8: Mendapatkan 10 ulasan. Total sejauh ini: 78
  -> Halaman 9: Mendapatkan 10 ulasan. Total sejauh ini: 88
  -> Halaman 10: Mendapatkan 10 ulasan. Total sejauh ini: 98
  -> Halaman 11: Mendapatkan 10 ulasan. Total sejauh ini: 108
  -> Halaman 12: Mendapatkan 10 ulasan. Total sejauh ini: 118
  -> Halaman 13: Mendapatkan 10 ulasan. Total sejauh ini: 128
  -> Halaman 14: Mendapatkan 10 ulasan. Total sejauh ini: 138
  -> Halaman 15: Mendapatkan 10 ulasan. Total sejauh i

--- REVIEW SCARPING & SAVE HASIL SCARPING ---

In [14]:
if all_reviews:
    print("\n Membersihkan dan memformat data ulasan...")

    # Buat list baru untuk menampung data yang sudah bersih
    cleaned_reviews_list = []
    for review in all_reviews:
        cleaned_reviews_list.append({
            'nama_penulis': review.get('user', {}).get('name'), # Akses nama dengan aman
            'rating': review.get('rating'),
            'teks_ulasan': review.get('snippet'),
            'waktu': review.get('date'),
            'likes': review.get('likes', 0)
        })

    # Buat DataFrame dari list yang sudah bersih
    df_final = pd.DataFrame(cleaned_reviews_list)

    # Simpan ke Colab
    df_final.to_csv(nama_file_output, index=False, encoding='utf-8-sig')
    print(f"\n SUKSES! {len(all_reviews)} ulasan telah disimpan dalam file '{nama_file_output}'")

    # Simpan ke Google Drive jika sudah di-mount
    try:
        drive.mount('/content/drive', force_remount=True)
        path_di_drive = f"/content/drive/MyDrive/{nama_file_output}"
        df_final.to_csv(path_di_drive, index=False, encoding='utf-8-sig')
        print(f" SUKSES! Data juga telah disimpan di Google Drive di lokasi: {path_di_drive}")
    except Exception as e:
        print(f"\n Gagal menyimpan ke Google Drive. Error: {e}")

    print("\nBerikut adalah 5 baris pertama data:")
    print(df_final.head())
else:
    print("\n Tidak ada ulasan yang ditemukan atau terjadi kesalahan pada proses sebelumnya.")



 Membersihkan dan memformat data ulasan...

 SUKSES! 2518 ulasan telah disimpan dalam file 'ulasan_manna_kampus_godean.csv'

 Gagal menyimpan ke Google Drive. Error: name 'drive' is not defined

Berikut adalah 5 baris pertama data:
         nama_penulis  rating  \
0         hilmy milan     5.0   
1                   O     5.0   
2  Dian Tri Lestarini     5.0   
3          ahmed jood     3.0   
4          Pasha Alif     5.0   

                                         teks_ulasan               waktu  \
0  This is a place that I like because firstly, t...        4 months ago   
1  The best thing about this place is "the cheap ...  Edited 8 years ago   
2  As someone who's new to living in Jogja, Manna...        6 months ago   
3  Always rush lot, things are good normal price ...         8 years ago   
4  Manna Kampus Godean is a hidden gem on Jalan G...          a year ago   

   likes  
0      0  
1      0  
2      0  
3      0  
4      0  


--- MENAMBAHKAN SENTIMEN ---

In [10]:
path_file_input = "/content/semua_ulasan_serpapi.csv"
nama_kolom_ulasan = 'teks_ulasan'
nama_file_output = 'hasil_ulasan_dengan_sentimen.csv'

print(f"\n Membaca file: {path_file_input}")

try:
    # Baca file CSV ke dalam DataFrame
    df = pd.read_csv(path_file_input)

    # Fungsi untuk analisis sentimen (Case Holding)
    def analisis_sentimen(teks):
        # Memastikan input adalah string
        if not isinstance(teks, str):
            return "Netral"

        # Analisis polaritas. Skor > 0.1 adalah positif, < -0.1 adalah negatif.
        if TextBlob(teks).sentiment.polarity > 0.1:
            return "Positif"
        elif TextBlob(teks).sentiment.polarity < -0.1:
            return "Negatif"
        else:
            return "Netral"

    print(f" Menerapkan analisis sentimen pada kolom '{nama_kolom_ulasan}'...")

    # Terapkan fungsi ke kolom ulasan dan buat kolom baru
    df['case_holding_sentimen'] = df[nama_kolom_ulasan].apply(analisis_sentimen)

    print(" Analisis sentimen selesai.")
    print("\n Berikut adalah 5 baris pertama data dengan kolom baru:")
    print(df.head())

    # Simpan DataFrame yang sudah diperbarui ke file CSV baru
    df.to_csv(nama_file_output, index=False, encoding='utf-8-sig')
    print(f"\n SUKSES! File baru telah disimpan dengan nama '{nama_file_output}'")

    #  Simpan juga hasilnya ke Google Drive
    try:
        drive.mount('/content/drive', force_remount=True)
        path_di_drive = f"/content/drive/MyDrive/{nama_file_output}"
        df_final.to_csv(path_di_drive, index=False, encoding='utf-8-sig')
        print(f" SUKSES! Data juga telah disimpan di Google Drive di lokasi: {path_di_drive}")
    except Exception as e:
        print(f"\n Gagal menyimpan ke Google Drive. Error: {e}")

except FileNotFoundError:
    print(f" ERROR: File '{path_file_input}' tidak ditemukan! Pastikan nama file sudah benar dan file sudah di-upload.")
except KeyError:
    print(f" ERROR: Kolom '{nama_kolom_ulasan}' tidak ditemukan di dalam file CSV! Periksa kembali nama kolom.")



 Membaca file: /content/semua_ulasan_serpapi.csv
 ERROR: File '/content/semua_ulasan_serpapi.csv' tidak ditemukan! Pastikan nama file sudah benar dan file sudah di-upload.
